# Final Project Template
**Notebook Preparation for Lesson**

Once you open the notebook:

1. Save in on your google drive (copy to drive) and name it DMAP FA21 Final Project
2. Share the notebook and copy the share ID to the NOTEBOOK_ID (and re-save the notebook)
3. This ID should remain the same for all your milestones
4. You will submit a copy of your updated notebook (this file) on Moodle for each milestone


In [1]:
# Keep this code cell here
# Project title will be used by the reviewer
PROJECT_TITLE = "Put your project title here"
NOTEBOOK_ID   = "https://colab.research.google.com/drive/1vzu2LxAkbpaArzj8nkQxfOXVkiTB98gd"
NET_ID = "junting6"
VERSION = "FA21"


---
# Project Introduction

Double click on this cell to see its contents.  We expect you to replace this cell.

<img align="left" src="http://drive.google.com/uc?export=view&id=1nA9491MchEtFcklvtIGqOnipE63C2FGD"/>

• Describe the **context**, sitution, problem to be solved/explored

• Whatever you need to get the reader _involved_

• Images can be hosted using google drive (you may need to create a transparent border)

• Even formulas when needed: 
$$e^x = \sum_{k=0}^{\infty} \frac{x^k}{k!}$$

• Note that markdown is very whitespace sensitive.
• Click on this cell to read comments.

<!-- this is a comment -->
<!-- 

   VIDEO INSTRUCTIONS (and data hosting)

1. upload to google drive, get the share URL
https://drive.google.com/file/d/1yGvY5a0KAqnOKf5kLh5EbbbRY4_LonAX

2. convert to export URL:
http://drive.google.com/uc?export=download&id=1yGvY5a0KAqnOKf5kLh5EbbbRY4_LonAX

3. OR use some other service to host your video:
https://storage.googleapis.com/uicourse/videos/dmap/Exact%20Instructions%20Challenge%20-%20THIS%20is%20why%20my%20kids%20hate%20me.%20%20Josh%20Darnit.mp4

replace the src="YOUR VIDEO URL" in the <source> tag in the next cell below
-->

In [2]:
# %%html
# <!-- this should be the ONLY html cell in the notebook: use markdown -->
# <div style="font-size:36px; max-width:800px; font-family:Times, serif;">
#  Title Your Video (and update the video URL)
# <video width="600" controls>
#   <source src="https://drive.google.com/uc?export=download&id=1yGvY5a0KAqnOKf5kLh5EbbbRY4_LonAX"
#   type="video/mp4">
# </video>
# </div>
# Note: If your video is too large, you can host it on Vimeo, YouTube, etc and paste the URL here

In [3]:
# add your imports here for your entire project
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
%matplotlib inline

# Project: Sentimental Message Probability

A detailed description about the problem/ research question(s) you are addressing, goals/expected outcome of the project, a brief introduction to your data, and the techniques you will be using.

### Introduction

Twitter, facebook, wechat etc. are common social media. People always communicate for some reason, either being professional or conveying feelings. As human, we can read and decipher what is the feeling embedded in a excerpt of text and perform corresponding reactions. However, can machine learning algorithms also "decipher" the embedded feelings within the text? 


### About dataset

the dataset I am using for this project is the sorted csv for centimental tweets. The source of this dataset can be found [here](https://www.kaggle.com/gargmanas/sentimental-analysis-for-tweets). This dataset labels each tweet with the indentifier as being_semtimental (as 1) and not_semtimental (as 0). 

the dataset has 3 columns, first one is index, second one is the text string of the message, third one is the identifier of whether this message is considered sentimental.

### About goal

The goal here I want to address is how to categorize a tweet/message as being semtimental or not. This is useful to find the embedded emotion based on the person's social activities, as well as we can expand this project to our other feelings like anger, joy, bitterness, etc. 

### What I hope to show and outline

I hope to train a model that can "read" a line of tweet and then categorize it as being sentimental or not. There are two initial thoughts about how to achieve this:

1. calculate the TFIDF of the sentimental corpus and the non sentimental corpus, then compare the words in the tweet with both models to classify this tweet to one of the category
2. train the gensim NLP model of the sentimental corpus and the not sentimental corpus, then compare the cosine similarities.
3. use supervised machine learning (something not covered yet in our class) to learn the difference of pattern in sentimental and not sentimental tweets, then test the model with incoming tweets.



In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/zhu-jun-ting/INFO-490-MH2/main/final_project/sentiment_tweets3.csv")

df

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0
...,...,...,...
10309,802309,No Depression by G Herbo is my mood from now o...,1
10310,802310,What do you do when depression succumbs the br...,1
10311,802311,Ketamine Nasal Spray Shows Promise Against Dep...,1
10312,802312,dont mistake a bad day with depression! everyo...,1


# Data Acquisition, Selection, Cleaning
Introduce the data here as well as any technical overview that's important that wasn't given in the introduction


# Data Exploration
Initial exploration of the dataset, not required, but useful to give the reader a 'view' of the data

# Data Analysis

# Summary


---
# Submission Guidelines (keep this section here)
---


When you are ready to submit your project, part of the submission process will be to register your notebook for reviewing.  

For each milestone, you will submit an updated version of your project notebook (this notebook) with that milestone’s requirements. Your project notebook ID and URL should be the same.

You will also receive the links and instructions to do the peer reviews.

**Submission for Milestone0:**
1. Save a copy of this notebook in your Google drive
3. Share the Noetbook and place the Notebook ID in the first code cell
4. Download as "DMAP_FA21_Final_Project_MS0.ipynb"
5. Submit this in Moodle
